## Recurrent Neural Networks

In this assignment, we will learn about recurrent neural networks. We will create an RNN and learn to classify text data.

In [18]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [19]:
yelp = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/yelp_labeled.csv', error_bad_lines=False)

b'Skipping line 281: expected 2 fields, saw 3\nSkipping line 290: expected 2 fields, saw 3\nSkipping line 296: expected 2 fields, saw 3\nSkipping line 322: expected 2 fields, saw 3\nSkipping line 373: expected 2 fields, saw 3\nSkipping line 417: expected 2 fields, saw 3\nSkipping line 427: expected 2 fields, saw 3\nSkipping line 429: expected 2 fields, saw 3\nSkipping line 577: expected 2 fields, saw 3\nSkipping line 578: expected 2 fields, saw 3\nSkipping line 611: expected 2 fields, saw 3\nSkipping line 677: expected 2 fields, saw 3\nSkipping line 771: expected 2 fields, saw 3\nSkipping line 930: expected 2 fields, saw 3\nSkipping line 979: expected 2 fields, saw 4\nSkipping line 980: expected 2 fields, saw 3\n'


In [20]:
yelp.head()

,text,sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [21]:
yelp.sentiment.value_counts()

1    494
0    482
Name: sentiment, dtype: int64

We have loaded a Yelp review dataset above. A positive sentiment is classified as 1 and a negative sentiment is classified as 0. 

In [22]:
from nltk.corpus import stopwords
import re
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()

def remove_stopwords(input_text):
        stopwords_list = stopwords.words('english')
        # Some words which might indicate a certain sentiment are kept via a whitelist
        whitelist = ["n't", "not", "no"]
        words = input_text.split() 
        clean_words = [word for word in words if (word not in stopwords_list or word in whitelist) and len(word) > 1] 
        return " ".join(clean_words)       

def stem_list(word_list):
    stemmed = []
    for word in word_list:
        stemmedword = stemmer.stem(word)
        stemmed.append(stemmedword)
    return stemmed

def normalize(terms):
    terms = terms.lower()
    terms = remove_stopwords(terms)
    word_delimiters = u'[\\[\\]\n.!?,;:\t\\-\\"\\(\\)\\\'\u2019\u2013 ]'
    term_list = re.split(word_delimiters, terms)
    trimmed = [x.rstrip() for x in term_list]
    stemmed = stem_list(trimmed)
    space = ' '
    normed = space.join(stemmed)
    normed = normed.replace('  ', ' ')
    return normed

In the code block above, we have functions to remove stopwords, stem, and normalize the text (remove special characters and trim white space). Apply the normalize function to every yelp review and assign the normalized text to a new column.

In [23]:
# Answer below:
yelp['norm_text'] = yelp['text'].apply(normalize)
yelp.head()

,text,sentiment,norm_text
0,Wow... Loved this place.,1,wow love place
1,Crust is not good.,0,crust not good
2,Not tasty and the texture was just nasty.,0,not tasti textur nasti
3,Stopped by during the late May bank holiday of...,1,stop late may bank holiday rick steve recommen...
4,The selection on the menu was great and so wer...,1,select menu great price


Next, use the one hot function for text encoding and encode the normalized text. Determine the vocabulary size to perform the encoding.

In [24]:
from nltk import word_tokenize
vocab_size = len(set(''.join(yelp.norm_text).split()))
vocab_size

1637

In [25]:
# Answer below:
from tensorflow.keras.preprocessing.text import one_hot

docs = yelp.norm_text

encoded_docs = [one_hot(doc, vocab_size) for doc in docs]
encoded_docs[:5]

[[1601, 123, 1627],
 [554, 1243, 1113],
 [1243, 983, 1402, 250],
 [779, 1229, 1073, 457, 894, 1002, 555, 96, 123, 1612],
 [205, 303, 821, 947]]

Convert the encoded sequences into a numpy array and make sure all reviews are the same length using the `pad_sequences` function in Keras.

In [26]:
# Answer below:
from tensorflow.keras.preprocessing.sequence import pad_sequences

ind_vars = pad_sequences(encoded_docs)
ind_vars[:5]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0, 1601,  123, 1627],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,

Split the data into train and test. Use 20% for test. The sentiment column should be used as the target variable.

In [27]:
# Answer below:
from sklearn.model_selection import train_test_split

y = yelp['sentiment']
X = ind_vars

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [28]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((780, 83), (196, 83), (780,), (196,))

Create a sequential model. The model should contain an embedding layer with input dim that is the size of the largest encoding in the vocabulary. The output dim should be 100, the input length is the number of columns in the training data. 
After the embedding layer, add a SimpleRNN layer with unit size 32, a dense layer of size 8 and a dense output layer.

In [29]:
ind_vars.shape

(976, 83)

In [30]:
# Answer below:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, SimpleRNN

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=100 , input_length=ind_vars.shape[1]))
model.add(SimpleRNN(32))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 83, 100)           163700    
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, 32)                4256      
_________________________________________________________________
dense_4 (Dense)              (None, 8)                 264       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 9         
Total params: 168,229
Trainable params: 168,229
Non-trainable params: 0
_________________________________________________________________


Compile using the optimizer of your choice, use crossentropy for your loss function. Fit the model using a batch size of 128 and 50 epochs

In [31]:
# Answer below:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(
    X_train,  
    y_train, 
    batch_size=128,  
    epochs=50, 
    validation_data=(X_test, y_test)
    )

Epoch 1/50
7/7 [==============================] - 2s 113ms/step - loss: 0.6882 - accuracy: 0.5437 - val_loss: 0.7009 - val_accuracy: 0.4439
Epoch 2/50
7/7 [==============================] - 1s 97ms/step - loss: 0.6526 - accuracy: 0.6283 - val_loss: 0.6941 - val_accuracy: 0.4592
Epoch 3/50
7/7 [==============================] - 0s 72ms/step - loss: 0.6111 - accuracy: 0.7198 - val_loss: 0.6905 - val_accuracy: 0.4694
Epoch 4/50
7/7 [==============================] - 1s 75ms/step - loss: 0.5629 - accuracy: 0.7481 - val_loss: 0.6452 - val_accuracy: 0.6122
Epoch 5/50
7/7 [==============================] - 1s 74ms/step - loss: 0.4790 - accuracy: 0.9227 - val_loss: 0.6039 - val_accuracy: 0.6939
Epoch 6/50
7/7 [==============================] - 1s 74ms/step - loss: 0.4039 - accuracy: 0.9544 - val_loss: 0.5408 - val_accuracy: 0.7500
Epoch 7/50
7/7 [==============================] - 0s 72ms/step - loss: 0.3336 - accuracy: 0.9597 - val_loss: 0.5371 - val_accuracy: 0.7347
Epoch 8/50
7/7 [==========